In [1]:
"""Libraries"""
import requests
import os
from dotenv import load_dotenv
import time
from datetime import datetime
import csv

In [2]:
URL = "https://opensky-network.org/api/states/all"
load_dotenv()
CLIENT_ID = os.getenv("clientId")
CLIENT_SECRET = os.getenv("clientSecret")
TOKEN_URL = "https://auth.opensky-network.org/auth/realms/opensky-network/protocol/openid-connect/token"

In [3]:
"""Asia region restricted"""
params = {
    "lamin": -10,
    "lamax": 55,
    "lomin": 25,
    "lomax": 180
}

In [4]:
"""Authentication"""
def get_access_token():
    r = requests.post(
        TOKEN_URL,
        headers = {"Content-Type": "application/x-www-form-urlencoded"},
        data = {
            "grant_type": "client_credentials",
            "client_id": CLIENT_ID,
            "client_secret": CLIENT_SECRET
        },
        timeout=10
    )
    r.raise_for_status()
    return r.json()["access_token"]

In [5]:
token = get_access_token()
token_time = time.time()

In [6]:
with open("./data/opensky_asia.csv", "a", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)

    writer.writerow([
        "timestamp",
        "icao24",
        "callsign",
        "latitude",
        "longitude",
        "baro_altitude",
        "velocity",
        "vertical_rate",
        "true_track",
        "on_ground"
    ])

    while True:
        # refresh token every 25 minutes
        if time.time() - token_time > 25 * 60:
            token = get_access_token()
            token_time = time.time()
            print("Token refreshed")

        headers = {
            "Authorization": f"Bearer {token}"
        }

        response = requests.get(
            URL,
            params=params,
            headers=headers,
            timeout=15
        )

        if response.status_code == 429:
            print("Rate limited (429). Backing off for 2 minutes.")
            time.sleep(120)
            continue

        if response.status_code != 200:
            print("Error:", response.status_code)
            time.sleep(30)
            continue

        data = response.json()

        if "time" not in data or "states" not in data:
            print("Unexpected response")
            time.sleep(30)
            continue

        timestamp = datetime.utcfromtimestamp(data["time"])

        count = 0
        for s in data["states"]:
            lat = s[6]
            lon = s[5]

            if lat is None or lon is None:
                continue

            writer.writerow([
                timestamp,
                s[0],      # icao24
                s[1],      # callsign
                lat,
                lon,
                s[7],      # baro_altitude
                s[9],      # velocity
                s[11],     # vertical_rate
                s[10],     # true_track
                s[8],      # on_ground
            ])
            count += 1

        print(f"{timestamp} | aircraft saved: {count}")
        time.sleep(10)

FileNotFoundError: [Errno 2] No such file or directory: './data/opensky_asia.csv'